# Flame Speed with Convergence Analysis

In this example we simulate a freely-propagating, adiabatic, 1-D flame and
* Calculate its laminar burning velocity
* Estimate the uncertainty in the laminar burning velocity calculation, due to grid size.

The figure below illustrates the setup, in a flame-fixed co-ordinate system. The reactants enter with density $\rho_{u}$, temperature $T_{u}$ and speed $S_{u}$. The products exit the flame at speed $S_{b}$, density $\rho_{b}$ and temperature $T_{b}$.

<img src="images/flameSpeed.png" alt="Freely Propagating Flame" style="width: 300px;"/>

### Import Modules

In [ ]:
import cantera as ct
import numpy as np
import pandas as pd

from IPython.display import display, HTML

import scipy
import scipy.optimize

print("Running Cantera Version: " + str(ct.__version__))

In [ ]:
# Import plotting modules and define plotting preference
%matplotlib inline
from matplotlib import pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
plt.rcParams['legend.fontsize'] = 10
plt.rcParams['figure.figsize'] = (8,6)

# Get the best of both ggplot and seaborn
plt.style.use('ggplot')
plt.style.use('seaborn-deep')

plt.rcParams['figure.autolayout'] = True

## Estimate uncertainty from grid size and speeds

In [ ]:
def extrapolate_uncertainty(grids, speeds, plot=True):
    """
    Given a list of grid sizes and a corresponding list of flame speeds,
    extrapolate and estimate the uncertainty in the final flame speed.
    Also makes a plot.
    """
    grids = list(grids)
    speeds = list(speeds)
    def speed_from_grid_size(grid_size, true_speed, error):
        """
        Given a grid size (or an array or list of grid sizes)
        return a prediction (or array of predictions)
        of the computed flame speed, based on 
        the parameters `true_speed` and `error`        
        """
        return true_speed +  error * np.array(grid_size)**-1.


    popt, pcov = scipy.optimize.curve_fit(speed_from_grid_size, grids[-4:], speeds[-4:])

    perr = np.sqrt(np.diag(pcov))
    true_speed_estimate  = popt[0]
    percent_error_in_true_speed = 100.*perr[0] / popt[0]
    print("Fitted true_speed is {:.4f} ± {:.4f} cm/s ({:.1f}%)".format(
        popt[0]*100,
        perr[0]*100,
        percent_error_in_true_speed
        ))
    #print "convergerce rate wrt grid size is {:.1f} ± {:.1f}".format(popt[2], perr[2])
    estimated_percent_error = 100. * (speed_from_grid_size(grids[-1], *popt) - true_speed_estimate) / true_speed_estimate
    print("Estimated error in final calculation {:.1f}%".format(estimated_percent_error))

    total_percent_error_estimate = abs(percent_error_in_true_speed) + abs(estimated_percent_error)
    print("Estimated total error {:.1f}%".format(total_percent_error_estimate))
    
    if not plot:
        return true_speed_estimate, total_percent_error_estimate

    plt.semilogx(grids,speeds,'o-')
    plt.ylim(min(speeds[-5:]+[true_speed_estimate])*.95, max(speeds[-5:]+[true_speed_estimate])*1.05)
    plt.plot(grids[-4:], speeds[-4:], 'or')
    extrapolated_grids = grids + [grids[-1] * i for i in range(2,8)]
    plt.plot(extrapolated_grids,speed_from_grid_size(extrapolated_grids,*popt),':r')
    plt.xlim(*plt.xlim())
    plt.hlines(true_speed_estimate, *plt.xlim(), colors=u'r', linestyles=u'dashed')

    plt.hlines(true_speed_estimate+perr[0], *plt.xlim(), colors=u'r', linestyles=u'dashed', alpha=0.3)
    plt.hlines(true_speed_estimate-perr[0], *plt.xlim(), colors=u'r', linestyles=u'dashed', alpha=0.3)
    plt.fill_between(plt.xlim(), true_speed_estimate-perr[0],true_speed_estimate+perr[0], facecolor='red', alpha=0.1 )

    #plt.text(grids[-1],speeds[-1],"{:.1f}%".format(estimated_percent_error))

    above = popt[1]/abs(popt[1]) # will be +1 if approach from above or -1 if approach from below
    
    plt.annotate("",
                 xy=(grids[-1], true_speed_estimate),
                 xycoords='data',
                 xytext=(grids[-1], speed_from_grid_size(grids[-1], *popt)),
                 textcoords='data',
                 arrowprops=dict(arrowstyle='|-|',
                                connectionstyle='arc3',
                                color='black', shrinkA=0, shrinkB=0),
                )
        
    plt.annotate("{:.1f}%".format(abs(estimated_percent_error)),
                 xy=(grids[-1], speed_from_grid_size(grids[-1], *popt)),
                 xycoords='data',
                 xytext=(10,20*above),
                 textcoords='offset points',
                 arrowprops=dict(arrowstyle='->',
                                connectionstyle='arc3')
                )
    
    plt.annotate("",
                 xy=(grids[-1]*4, true_speed_estimate-(above*perr[0])),
                 xycoords='data',
                 xytext=(grids[-1]*4, true_speed_estimate),
                 textcoords='data',
                 arrowprops=dict(arrowstyle='|-|',
                                connectionstyle='arc3',
                                color='black', shrinkA=0, shrinkB=0),
                )
    plt.annotate("{:.1f}%".format(abs(percent_error_in_true_speed)),
                 xy=(grids[-1]*4, true_speed_estimate-(above*perr[0])),
                 xycoords='data',
                 xytext=(10,-20*above),
                 textcoords='offset points',
                 arrowprops=dict(arrowstyle='->',
                                connectionstyle='arc3')
                )

    plt.ylabel("Flame speed (m/s)")
    plt.xlabel("Grid size")
    plt.show()
    
    return true_speed_estimate, total_percent_error_estimate

In [ ]:
def make_callback(flame):
    speeds = []
    grids = []

    def callback(_):
        speed = flame.u[0]
        grid = len(flame.grid)
        speeds.append(speed)
        grids.append(grid)
        print("Iteration {}".format(len(grids)))
        print("Current flame speed is is {:.4f} cm/s".format(speed*100.))
        if len(grids) < 5:
            return 1.0 # 
        try:
            extrapolate_uncertainty(grids, speeds)
        except Exception as e:
            print("Couldn't estimate uncertainty. " + str(e))
            return 1.0 # continue anyway

        return 1.0
    return callback, speeds, grids


### Define the reactant conditions, gas mixture and kinetic mechanism associated with the gas

In [ ]:
#Inlet Temperature in Kelvin and Inlet Pressure in Pascals
#In this case we are setting the inlet T and P to room temperature conditions
To = 300
Po = 101325

#Define the gas-mixutre and kinetics
#In this case, we are choosing a GRI3.0 gas
gas = ct.Solution('input-files/gri30_noNOx.cti')

# Create a stoichiometric CH4/Air premixed mixture 
gas.set_equivalence_ratio(1.0, 'CH4', {'O2':1.0, 'N2':3.76})
gas.TP = To, Po

### Define flame simulation conditions

In [ ]:
# Domain width in metres
width = 0.014

# Create the flame object
flame = ct.FreeFlame(gas, width=width)

# Define logging level
loglevel = 1

In [ ]:
# Define tight tolerances for the solver
refine_criteria = {'ratio':2, 'slope': 0.01, 'curve': 0.01}
flame.set_refine_criteria(**refine_criteria)

# Set maxiumum number of grid points to be very high (otherwise default is 1000)
flame.set_max_grid_points(flame.domains[flame.domain_index('flame')], 1e4)

# Set up the the callback function and lists of speeds and grids
callback, speeds, grids = make_callback(flame)
flame.set_steady_callback(callback)

# Define logging level
loglevel = 1

### Solve

In [ ]:
flame.solve(loglevel=loglevel, auto=True)

Su0 = flame.u[0]
print("Flame Speed is: {:.2f} cm/s".format(Su0*100))


In [ ]:
best_true_speed_estimate, best_total_percent_error_estimate =  extrapolate_uncertainty(grids, speeds)

In [ ]:
len(grids)

In [ ]:
def analyze_errors(grids, speeds):
    true_speed_estimates = [None for i in speeds]
    total_percent_error_estimates = [None for i in speeds]
    actual_extrapolated_percent_errors = [None for i in speeds]
    actual_raw_percent_errors = [None for i in speeds]
    for i in range(3,len(grids)):
        print(grids[:i+1])
        true_speed_estimate, total_percent_error_estimate = extrapolate_uncertainty(grids[:i+1], speeds[:i+1], plot=False)
        actual_extrapolated_percent_error = 100. * abs(true_speed_estimate - best_true_speed_estimate) / best_true_speed_estimate
        actual_raw_percent_error = 100. * abs(speeds[i] - best_true_speed_estimate) / best_true_speed_estimate
        print("Actual extrapolated error (with hindsight) {:.1f}%".format(actual_extrapolated_percent_error))
        print("Actual raw error (with hindsight) {:.1f}%".format(actual_raw_percent_error))

        true_speed_estimates[i] = true_speed_estimate
        total_percent_error_estimates[i] = total_percent_error_estimate
        actual_extrapolated_percent_errors[i] = actual_extrapolated_percent_error
        actual_raw_percent_errors[i] = actual_raw_percent_error
        print()
        
    plt.loglog(grids, actual_raw_percent_errors,'o-', label='raw error')
    plt.loglog(grids, actual_extrapolated_percent_errors,'o-', label='extrapolated error')
    plt.loglog(grids, total_percent_error_estimates,'o-', label='estimated error')
    plt.ylabel("Error in flame speed (%)")
    plt.xlabel("Grid size")
    plt.legend()
    plt.title(flame.get_refine_criteria())
    plt.show()
    flame.get_refine_criteria()
    
    data = pd.DataFrame(data={'actual error in raw value': actual_raw_percent_errors,
                   'actual error in extrapolated value':actual_extrapolated_percent_errors,
                   'estimated error':total_percent_error_estimates}, 
              index=grids)
    display(data)


In [ ]:
analyze_errors(grids, speeds)

## Middling refine criteria

In [ ]:
refine_criteria = {'ratio':3, 'slope': 0.1, 'curve': 0.1}

In [ ]:
# Reset the gas
gas.set_equivalence_ratio(1.0, 'CH4', {'O2':1.0, 'N2':3.76})
gas.TP = To, Po

# Create a new flame object
flame = ct.FreeFlame(gas, width=width)

flame.set_refine_criteria(**refine_criteria)
flame.set_max_grid_points(flame.domains[flame.domain_index('flame')], 1e4)

callback, speeds, grids = make_callback(flame)
flame.set_steady_callback(callback)

# Define logging level
loglevel = 1

flame.solve(loglevel=loglevel, auto=True)

Su0 = flame.u[0]
print("Flame Speed is: {:.2f} cm/s".format(Su0*100))

In [ ]:
analyze_errors(grids, speeds)

## Default (loose) criteria

In [ ]:
flame = ct.FreeFlame(gas, width=width)
flame.get_refine_criteria()
refine_criteria = flame.get_refine_criteria()
refine_criteria.update({'prune':0})
refine_criteria

In [ ]:
gas.set_equivalence_ratio(1.0, 'CH4', {'O2':1.0, 'N2':3.76})
gas.TP = To, Po

# Create a new flame object
flame = ct.FreeFlame(gas, width=width)

flame.set_refine_criteria(**refine_criteria)
flame.set_max_grid_points(flame.domains[flame.domain_index('flame')], 1e4)

callback, speeds, grids = make_callback(flame)
flame.set_steady_callback(callback)

# Define logging level
loglevel = 1

flame.solve(loglevel=loglevel, auto=True)

Su0 = flame.u[0]
print("Flame Speed is: {:.2f} cm/s".format(Su0*100))

In [ ]:
analyze_errors(grids, speeds)

## Middling refine criteria

In [ ]:
refine_criteria = {'ratio':3, 'slope': 0.1, 'curve': 0.1}

In [ ]:
# Reset the gas
gas.set_equivalence_ratio(1.0, 'CH4', {'O2':1.0, 'N2':3.76})
gas.TP = To, Po

# Create a new flame object
flame = ct.FreeFlame(gas, width=width)

flame.set_refine_criteria(**refine_criteria)
flame.set_max_grid_points(flame.domains[flame.domain_index('flame')], 1e4)

callback, speeds, grids = make_callback(flame)
flame.set_steady_callback(callback)

# Define logging level
loglevel = 1

flame.solve(loglevel=loglevel, auto=True)

Su0 = flame.u[0]
print("Flame Speed is: {:.2f} cm/s".format(Su0*100))

In [ ]:
analyze_errors(grids, speeds)

### Plot figures

Check and see if all has gone well. Plot temperature and species fractions to see

#### Temperature Plot

In [ ]:
plt.figure()

plt.plot(flame.grid*100, flame.T, '-o')
plt.xlabel('Distance (cm)')
plt.ylabel('Temperature (K)');

#### Major species' plot

In [ ]:
"""
# To plot species, we first have to identify the index of the species in the array
# For this, cut & paste the following lines and run in a new cell to get the index
for i, specie in enumerate(gas.species()):
    print(str(i) + '. ' + str(specie))
"""

# Extract concentration data
X_CH4 = flame.X[13]
X_CO2 = flame.X[15]
X_H2O = flame.X[5]

plt.figure()

plt.plot(flame.grid*100, X_CH4, '-o', label=r'$CH_{4}$')
plt.plot(flame.grid*100, X_CO2, '-s', label=r'$CO_{2}$')
plt.plot(flame.grid*100, X_H2O, '-<', label=r'$H_{2}O$')

plt.legend(loc=2)
plt.xlabel('Distance (cm)')
plt.ylabel('MoleFractions');

## Try a Hydrogen flame (still with GRI mech)

In [ ]:
# Tight criteria
refine_criteria = {'ratio':2, 'slope': 0.01, 'curve': 0.01}

In [ ]:
# Reset the gas
gas.set_equivalence_ratio(1.0, 'H2', {'O2':1.0, 'N2':3.76})
gas.TP = To, Po

# Create a new flame object
flame = ct.FreeFlame(gas, width=width)

flame.set_refine_criteria(**refine_criteria)
flame.set_max_grid_points(flame.domains[flame.domain_index('flame')], 1e4)

callback, speeds, grids = make_callback(flame)
flame.set_steady_callback(callback)

# Define logging level
loglevel = 1

flame.solve(loglevel=loglevel, auto=True)

Su0 = flame.u[0]
print("Flame Speed is: {:.2f} cm/s".format(Su0*100))

In [ ]:
best_true_speed_estimate, best_total_percent_error_estimate =  extrapolate_uncertainty(grids, speeds)

In [ ]:
analyze_errors(grids, speeds)

## Middling refine criteria, Hydrogen flame

In [ ]:
refine_criteria = {'ratio':3, 'slope': 0.1, 'curve': 0.1}

In [ ]:
# Reset the gas
gas.set_equivalence_ratio(1.0, 'H2', {'O2':1.0, 'N2':3.76})
gas.TP = To, Po

# Create a new flame object
flame = ct.FreeFlame(gas, width=width)

flame.set_refine_criteria(**refine_criteria)
flame.set_max_grid_points(flame.domains[flame.domain_index('flame')], 1e4)

callback, speeds, grids = make_callback(flame)
flame.set_steady_callback(callback)

# Define logging level
loglevel = 1

flame.solve(loglevel=loglevel, auto=True)

Su0 = flame.u[0]
print("Flame Speed is: {:.2f} cm/s".format(Su0*100))

In [ ]:
analyze_errors(grids, speeds)